In [ ]:
%load_ext autoreload
%autoreload 2

from scribbles_creator import *
from PIL import Image
import napari
import numpy as np

## Create scribble annotation from an image mask of the cell pose dataset

In [ ]:
# Read the ground truth as an image
img_num = 1
image_path = f"./imgs/train/{str(img_num).zfill(3)}"
ground_truth = np.array(Image.open(image_path + "_masks.png"))
# Summarize all non-background classes into one class (we are testing semantic segmentation not instance segmentation)
ground_truth[ground_truth>0] = 2
# Set the background class to 1 (since the scribble annotation assumes class 0 to represent non-annotated pixels)
ground_truth[ground_truth==0] = 1

# Generate the scribble annotation for the ground truth
scribbles = create_scribble(ground_truth, scribble_width=1, num_squares=5, sq_size=20, min_sq_pix=10, num_lines=5, min_line_pix=10, mode="all")
perc_labelled = np.sum(scribbles>0) / (scribbles.shape[0] * scribbles.shape[1]) * 100
print(f"Percentage of labelled pixels: {perc_labelled:.2f}%")

## Save or show the scribbles and ground truth

In [ ]:
# Save the scribbles and ground truth image to a file (first, convert the numpy arrays to images)
scribbles_image = Image.fromarray(scribbles)
ground_truth_image = Image.fromarray(ground_truth)
scribbles_image.save(image_path + '_scribbles.png')
ground_truth_image.save(image_path + '_ground_truth.png')

In [ ]:
# Show the image, ground truth and the scribble annotation in a napari viewer
v = napari.Viewer()
v.add_labels(ground_truth)
v.add_labels(scribbles)
image = np.array(Image.open(image_path + "_img.png"))
v.add_image(image)

## When we have performed segmentation in Napari, we can calculate its accuracy

In [ ]:
segmentation = v.layers['segmentation'].data
acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
print(f"Accuracy (pixels correctly segmented): {acc}%")